This notebook has been used to find the optimized time complexitites.

We have to run the below cell for all instantiations described in the following cells.

In [1]:
import collections
import random
from random import  uniform
from math import *
import scipy.optimize as opt

def H(c):
    """
    Entropy function
    """
    if c == 0. or c == 1.:
        return 0.
    
    if c < 0. or c > 1.:
        return -1000
    
    return -(c * log2(c) + (1 - c) * log2(1 - c))

def binomH(n,k):
    """
    binomial coefficient
    """
    # if k/n not in ZZ:
    #     return -100
    if(n<=0):
        return 0.
    return n * H(k/n)

def multiH(n,c):
    """
    multinomial coefficient
    """
    if sum(c)>n:
        return 0
    tot=0
    val=n
    for i in c:
        tot+=binomH(n,i)
        n-=i
    return tot


def wrap(f,g) :
    def inner(x):
        return f(g(*x))
    return inner

def r(x,y,z):
    return [(ru(x,y)) for i in range(z)]

For van Vredendaal's instantiation.

In [61]:
def time(w):
    T1 = multiH(1/2,[w/4,w/4])
    TvV = T1*(3/2)
    return TvV

for i in range(25,26):
    w = .02*i
    print((round(w,2) , round(time(w),4)))

(0.5, 1.125)


For May's Rep-0 instantiation.

In [62]:
def time(w):
    T1 = multiH(1,[w/4,w/4])
    R = 2*binomH(w/2,w/4)
    Trep0 = T1*(3/2)-R
    return Trep0

for i in range(25,26):
    w = .02*i
    print((round(w,2) , round(time(w),4)))

(0.5, 1.0919)


For May's Rep-1 instantiation.

In [63]:
set_vars = collections.namedtuple('LWE', 'd')
def lwe(f) : return wrap(f, set_vars)

def time(x):
    x=set_vars(*x)
    T1 = multiH(1,[w/4+x.d,w/4+x.d])
    R = 2*binomH(w/2,w/4) + multiH(1-w,[x.d,x.d])
    Trep1 = 1.5*T1 - R
    return Trep1

constraints_lwe = [
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -2*x.d+1-w)},
]

start=[.001]
bounds=[(0,0.1)]
for i in range(25,26):
    w = i*.02
    result = opt.minimize(time, start, 
            bounds= bounds, tol=1e-12, 
            constraints=constraints_lwe, options={'maxiter':2000})
    r = result.get('fun')
    print((round(w,4),round(r,4)))

(0.5, 0.9635)


For May's Rep-2 instantiation.

In [64]:
set_vars = collections.namedtuple('LWE', 'e21 e20 e10')
def lwe(f) : return wrap(f, set_vars)

n2 = lambda x :x.e21+x.e20
n1 = lambda x :w/4 + x.e10
n0 = lambda x :1-w/2-2*(x.e10+x.e21+x.e20)


def time(x):
    x=set_vars(*x)
    T1 = multiH(1,[n2(x),n2(x),n1(x),n1(x)])
    R = 2*multiH(w/2,[x.e21,x.e21,w/4-x.e21]) + multiH(1-w,[x.e20,x.e20,x.e10,x.e10])
    Trep1 = 1.5*T1 - R
    return Trep1


constraints_lwe = [
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.e21+w/4)},   
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.e10-x.e20+1/2-w/2)},   

]


res = 100
bounds=[(0,0.1)]*3
for i in range(25,26):
    w = i*.02
    res = 100
    for j in range(10):
        start = [uniform(0,.009), uniform(0,.009), uniform(0,.009)]
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-12, 
                constraints=constraints_lwe, options={'maxiter':2000})
        r = result.get('fun')
        if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
            res = r
    print((round(w,4),round(res,4)))
    
    
    

(0.5, 0.962)


For Nested-1+

In [65]:
def reps(p, m, d, l): 
    """
    representations of length-l vector with p ones and m minus ones
    two length-l vectors with p/2+d ones and d minus ones each.
    """
    if p <= 0.000001 or l == 0.:
        return 0
    if l < p or l - p < 2*d:
        return 0.
    
    return p + m + multiH(l-p-m, [d,d]) 


set_vars = collections.namedtuple('LWE', 'd1 d2 g') 
def lwe(f) : return wrap(f, set_vars)

def time(x):
    x=set_vars(*x)
    good_R = max(0,domain(x) - reps(x.g*w/2,x.g*w/2,x.d2,x.g))                        #inverse of q_{2}
    collisions_when_R_good = domain(x) - 2*reps(x.g*w/4+x.d2,x.g*w/4+x.d2,x.d1,x.g)   #inverse of q_{1}
    one_collision=domain(x)
    return good_R+collisions_when_R_good+one_collision


#domain defines the function domain size.
domain = lambda x : multiH(x.g,[(x.g*w/4+x.d2)*.5+x.d1,(x.g*w/4+x.d2)*.5+x.d1]) + multiH((1-x.g)*.25,[(1-x.g)*w*.125,(1-x.g)*w*.125])

constraints_lwe = [
# domain must be equal to 1/2
{ 'type' : 'eq',   'fun' : lwe(lambda x : multiH(1,[w/2,w/2])*.5-domain(x))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - reps(x.g*w/2,x.g*w/2,x.d2,x.g))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - 2*reps(x.g*w/4+x.d2,x.g*w/4+x.d2,x.d1,x.g))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : x.g-x.g*w-2*x.d2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : x.g-x.g*w/2-2*x.d2-2*x.d1)}

]


bounds=[(0,0.1)]*2+[(0.01,1)]
for i in range(25,26):
    w = i*.02
    res = 100
    for j in range(4):
        start = [uniform(0,.009), uniform(0,.009), uniform(0.1,1)]  
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-12, 
                constraints=constraints_lwe, options={'maxiter':2000})
        r = result.get('fun')
        if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
            res = r
    print((round(w,4),round(res,4))) 

(0.5, 0.8529)


For Nested-1*

In [66]:
def reps(p, m, d, l): 
    """
    representations of length-l vector with p ones and m minus ones
    two length-l vectors with p/2+d ones and d minus ones each.
    """
    if p <= 0.000001 or l == 0.:
        return 0
    if l < p or l - p < 2*d:
        return 0.
    
    return p + m + multiH(l-p-m, [d,d]) 


set_vars = collections.namedtuple('LWE', 'd1 d2 g b')
def lwe(f) : return wrap(f, set_vars)


p2 = lambda x : x.b*x.g*w/4+x.d2     #number of \pm1 in the \gamma n part in each summand in mid-level 

p1 = lambda x : p2(x)/2+x.d1         #number of \pm1 in the \gamma n part in each summand in base-level


def time(x):
    x=set_vars(*x)
    good_D = multiH(1,[w/2,w/2]) - ss(x)
    good_R = max(0,domain(x) - reps(x.b*x.g*w/2,x.b*x.g*w/2,x.d2,x.g))
    collisions_when_R_good = domain(x) - 2*reps(p2(x),p2(x),x.d1,x.g)
    one_collision=domain(x)
    return good_R+collisions_when_R_good+one_collision+good_D



domain = lambda x : multiH(x.g,[p1(x),p1(x)]) + multiH((1-x.g)*.25,[(1-x.b*x.g)*w*.125,(1-x.b*x.g)*w*.125])
# 'ss' gives the search space which is denoted by 'D' in our paper.
ss = lambda x : multiH(x.g,[x.g*x.b*w/2, x.g*x.b*w/2]) + 4*multiH((1-x.g)/4,[(1-x.g*x.b)*w*.125,(1-x.g*x.b)*w*.125])

constraints_lwe = [
# domain must be equal to 1/2
{ 'type' : 'eq',   'fun' : lwe(lambda x : ss(x)*.5-domain(x))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - reps(x.b*x.g*w/2,x.b*x.g*w/2,x.d2,x.g))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - 2*reps(p2(x),p2(x),x.d1,x.g))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : x.b - (1/x.g)+(1/(x.g*w))-(1/w))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : 1-x.b)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x :  multiH(1,[w/2,w/2]) - ss(x))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : x.g-x.b*x.g*w-2*x.d2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : x.g-x.b*x.g*w/2-2*x.d2-2*x.d1)}

]


bounds=[(0,0.1)]*2+[(0.01,1)]*2
res = 100
for i in range(25, 26):
    w = i*.02
    res = 100
    for j in range(5):
        start = [uniform(0,.009), uniform(0,.009), uniform(0.1,1), uniform(0.1,1)]        
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-12, 
                constraints=constraints_lwe, options={'maxiter':2000})
        #print(result)
        r = result.get('fun')
        if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
            res = r
    print((round(w,4),round(res,4)))
    

(0.5, 0.8505)


For Nested-2*

In [67]:
def reps1(wcap1,wca0,omid,zmid2,zmid1,p): 
   #length of whole vec = p 
    if wcap1 <= 0.000001 or p == 0.:
        return 0
    if p < wcap1 or p - 2*wcap1 < 2*zmid1:
        return 0.
    
    return 2*multiH(wcap1, [omid,omid,wcap1*.5-omid]) + multiH(p-2*wcap1,[zmid2,zmid2,zmid1,zmid1])


def reps2(nmid1,nmid0,nmid2,t_bar,o,z2,z1,p): 
    
    if nmid1 < 0.000001 or p == 0.:
        return 0.
    if p < nmid1 or p<nmid2 or p - 2*nmid1 - nmid2 < 2*z1:
        return 0.
    
    return 2*multiH(nmid2,[t_bar,t_bar])+2*multiH(nmid1, [o, o, nmid1*.5-o])+multiH(nmid0, [z2, z2, z1, z1])



set_vars = collections.namedtuple('LWE', ' omid zmid2 zmid1 t_bar o z2 z1 g b')
'''
t_bar  = (nmid2-t)/2
'''
def lwe(f) : return wrap(f, set_vars)


wcap1 = lambda x : x.b*x.g*w/2               #number of \pm1 in the \gamma n part in the final solution. wcap1 = wcap/2
wca0 = lambda x : x.g*1 - 2*wcap1(x)        #number of 0s in the \gamma n part in the final solution 

nmid2 = lambda x : x.omid+x.zmid2              #number of \pm1 in the \gamma n part in each summand in mid-level
nmid1 = lambda x : wcap1(x)/2+x.zmid1           #number of \pm2 in the \gamma n part in each summand in mid-level
nmid0 = lambda x : x.g*1-wcap1(x)-2*(x.zmid1+x.omid+x.zmid2)   #number of 0s in the \gamma n part in each summand in mid-level

n2 = lambda x : x.t_bar+x.o+x.z2
n1 = lambda x : nmid1(x)/2+x.z1+nmid2(x)-2*x.t_bar
n0 = lambda x : x.g*1-nmid1(x)-2*nmid2(x)+2*(x.t_bar-x.z1-x.o-x.z2)


def time(x):
    x=set_vars(*x)
    good_D = multiH(1,[w/2,w/2]) - ss(x)
    good_R = max(0,domain(x) - reps1(wcap1(x),wca0(x),x.omid,x.zmid2,x.zmid1,x.g*1))
    collisions_when_R_good = domain(x) - 2*reps2(nmid1(x),nmid0(x),nmid2(x),x.t_bar,x.o,x.z2,x.z1,x.g*1)
    one_collision=domain(x)
    return good_R+collisions_when_R_good+one_collision+good_D




domain = lambda x : multiH(x.g*1,[n2(x), n2(x), n1(x), n1(x)])+ multiH((1-x.g)*.25,[(1-x.b*x.g)*w*.125,(1-x.b*x.g)*w*.125])
ss = lambda x : multiH(x.g,[x.g*x.b*w/2, x.g*x.b*w/2]) + 4*multiH((1-x.g)/4,[(1-x.g*x.b)*w*.125,(1-x.g*x.b)*w*.125])
# 'ss' gives the search space which is denoted by 'D' in our paper.

constraints_lwe = [
# domain muss be equal to 1/2
    
{ 'type' : 'eq',   'fun' : lwe(lambda x : ss(x)/2-domain(x))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - reps1(wcap1(x),wca0(x),x.omid,x.zmid2,x.zmid1,x.g*1))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - 2*reps2(nmid1(x),nmid0(x),nmid2(x),x.t_bar,x.o,x.z2,x.z1,x.g*1))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.omid+wcap1(x)/2)},
# { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.zmid2+wca0(x)/2)},
# { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.zmid1+wca0(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.zmid1-x.zmid2+wca0(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.t_bar+nmid2(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o+nmid1(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2-x.z1+nmid0(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : x.b - (1/x.g)+(1/(x.g*w))-(1/w))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : 1-x.b)}, 
{ 'type' : 'ineq',   'fun' : lwe(lambda x :  multiH(1,[w/2,w/2]) - ss(x))},

]

    
for i in range(25, 26):
    w = i*.02
    res = 100
    ress = 5000
    for j in range(5):
        start=[uniform(0,.009), uniform(0,.009), uniform(0,.009),uniform(0,.009),uniform(0,.009),uniform(0,.009),uniform(0,.009), uniform(.01,1), uniform(.01,1)]
        
        bounds=[(0.,0.1)]*7+[(.01,1)]*2
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-12, 
                constraints=constraints_lwe, options={'maxiter':2000})
        r = result.get('fun')
        if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
               res = r
               ress = result
    print((round(w,4),round(res,4)))   

(0.5, 0.8425)


For Uniform Secret Variation

In [4]:
def reps1(wcap1,wca0,omid,zmid2,zmid1,p): 
   #length of whole vec = p 
    if wcap1 <= 0.000001 or p == 0.:
        return 0
    if p < wcap1 or p - 2*wcap1 < 2*zmid1:
        return 0.
    
    return 2*multiH(wcap1, [omid,omid,wcap1*.5-omid]) + multiH(p-2*wcap1,[zmid2,zmid2,zmid1,zmid1])


def reps2(nmid1,nmid0,nmid2,t_bar,o,z2,z1,p): 
    
    if nmid1 < 0.000001 or p == 0.:
        return 0.
    if p < nmid1 or p<nmid2 or p - 2*nmid1 - nmid2 < 2*z1:
        return 0.
    
    return 2*multiH(nmid2,[t_bar,t_bar])+2*multiH(nmid1, [o, o, nmid1*.5-o])+multiH(nmid0, [z2, z2, z1, z1])


set_vars = collections.namedtuple('LWE', ' omid zmid2 zmid1 t_bar o z2 z1 g b p')
def lwe(f) : return wrap(f, set_vars)


wcap1 = lambda x : x.b*x.g*omega(x)/2
wca0 = lambda x : x.g*1 - 2*wcap1(x)

nmid2 = lambda x : x.omid+x.zmid2
nmid1 = lambda x : wcap1(x)/2+x.zmid1
nmid0 = lambda x : x.g*1-wcap1(x)-2*(x.zmid1+x.omid+x.zmid2)

n2 = lambda x : x.t_bar+x.o+x.z2
n1 = lambda x : nmid1(x)/2+x.z1+nmid2(x)-2*x.t_bar
n0 = lambda x : x.g*1-nmid1(x)-2*nmid2(x)+2*(x.t_bar-x.z1-x.o-x.z2)


def time(x):
    x=set_vars(*x)
    if x.p<0.000001:
        Tp=0
    else:
        Tp=perm2(omega(x),x.p)
    good_D = multiH(1,[omega(x)/2,omega(x)/2]) - ss(x)
    good_R = max(0,domain(x) - reps1(wcap1(x),wca0(x),x.omid,x.zmid2,x.zmid1,x.g*1))
    collisions_when_R_good = domain(x) - 2*reps2(nmid1(x),nmid0(x),nmid2(x),x.t_bar,x.o,x.z2,x.z1,x.g*1)
    one_collision=domain(x)
    return good_R+collisions_when_R_good+one_collision+good_D+Tp



def perm2(w,p):
    return multiH(2,[(w+p)/2+1/3,(w+p)/2+1/3])-(multiH(1,[(w)/2,(w)/2])+multiH(1,[1/3+p/2,1/3+p/2]))

domain = lambda x : multiH(x.g*1,[n2(x), n2(x), n1(x), n1(x)])+ multiH((1-x.g)*.25,[(1-x.b*x.g)*omega(x)*.125,(1-x.b*x.g)*omega(x)*.125])
ss = lambda x : multiH(x.g,[x.g*x.b*omega(x)/2, x.g*x.b*omega(x)/2]) + 4*multiH((1-x.g)/4,[(1-x.g*x.b)*omega(x)*.125,(1-x.g*x.b)*omega(x)*.125])
# 'ss' gives the search space which is denoted by 'D' in our paper.

constraints_lwe = [
# domain muss be equal to 1/2
    
{ 'type' : 'eq',   'fun' : lwe(lambda x : ss(x)/2-domain(x))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - reps1(wcap1(x),wca0(x),x.omid,x.zmid2,x.zmid1,x.g*1))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - 2*reps2(nmid1(x),nmid0(x),nmid2(x),x.t_bar,x.o,x.z2,x.z1,x.g*1))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.omid+wcap1(x)/2)},
# { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.zmid2+wca0(x)/2)},
# { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.zmid1+wca0(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.zmid1-x.zmid2+wca0(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.t_bar+nmid2(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o+nmid1(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2-x.z1+nmid0(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : x.b - (1/x.g)+(1/(x.g*omega(x)))-(1/omega(x)))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : 1-x.b)}, 
{ 'type' : 'ineq',   'fun' : lwe(lambda x :  multiH(1,[omega(x)/2,omega(x)/2]) - ss(x))},

]

    
for i in range(25,26):
    w = 2/3 #i*0.02
    omega = lambda x: w-x.p
    res = 100
    ress = 5000
    for j in range(5):
        start=[uniform(0,.009), uniform(0,.009), uniform(0,.009),uniform(0,.009),uniform(0,.009),uniform(0,.009),uniform(0,.009), uniform(.01,1), uniform(.01,1), uniform(0,.1)]
        
        bounds=[(0.,0.1)]*7+[(.01,1)]*2+[(0,1)]
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-12, 
                constraints=constraints_lwe, options={'maxiter':250})
        #print(result)
        r = result.get('fun')
        if r < res and r >0 and result.success:
            res = r
            ress = result
    print((round(w,4),round(res,4)))   

(0.6667, 0.926)
